In [2]:
from scipy import signal
def calculateCFO(fileDir, sigReal, sigImag):
    sampleRate = 112*(10**6)
    shortestMsg = 8*(10**-6)
    minSamples = int(sampleRate * shortestMsg)
    print(minSamples)
    print(fileDir)
    
    df = pd.read_excel(fileDir)
    preamble = df.head(minSamples)
    
    complexNumbers = np.vectorize(complex)(preamble['real'], preamble['imaginary'])
    
    ampl = np.absolute(complexNumbers)
    angle = np.angle(complexNumbers)
    
    I = np.dot(ampl, np.cos(angle))
    Q = np.dot(ampl, np.sin(angle))
    
    Signal_I = np.dot(sigReal, I)
    Signal_Q = np.dot(sigImag, Q)
    
    return Signal_I, Signal_Q
    
    

    

In [3]:
def callCalculate(directory):
    
    sampleRate = 112*(10**6)
    shortestMsg = 8*(10**-6)
    channelFreq = 28*(10**6)
    minSamples = sampleRate * shortestMsg
    dt = 1/sampleRate
    
    steps = []
    i = 0
    
    while i <= shortestMsg:
        steps.append(i)
        i += dt
    
    sigReal = np.cos(2*math.pi*channelFreq*np.array(steps))
    sigImag = np.sin(2*math.pi*channelFreq*np.array(steps))
    
    bursts = [f for f in listdir(directory) if isfile(join(directory, f))]
    
    for file in bursts:
        fileDir = directory + slash + file
        Signal_I, Signal_Q = calculateCFO(fileDir, sigReal, sigImag)
        filterSignals(Signal_I, Signal_Q)
        
        
        


In [4]:
def filterSignals(Signal_I, Signal_Q):
    
    fCutoff = 20*(10**6)
    sampleRate = 112*(10**6)
    b, a = signal.butter(8, fCutoff/(sampleRate/2))
    filterI = signal.lfilter(b,a,Signal_I) # Double check that this is not low pass filter. seems to be right...
    filterQ = signal.lfilter(b,a,Signal_Q)
    signalBaseband = np.vectorize(complex)(filterI, filterQ)
    phaseReal = np.angle(signalBaseband)
    unwrappedPhaseReal = np.unwrap(phaseReal)
    average = []
    
    
    # Find First 8 symbols
    for x in range(1, 9):
        pos = 0
        difference = []
        for i in range((x * 112), ((x-1)*112) + 1, -1):
            difference.append(np.absolute(unwrappedPhaseReal[i] - unwrappedPhaseReal[i-1]))
            pos += 1
#         print(len(difference))
        
        average.append(np.mean(difference))
    mySum = sum(average)
    f_offset = mySum/(2*math.pi*x*(2*(10**-6)))
    print(f_offset)

In [10]:
# This program is meant to help generate Carrier Frequency Offset data
# from IQ data. I am basing this code on work done by Peter Ekpewoh in 
# Matlab.
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np

if __name__ == '__main__':
    path = "smallBursts/Fever Sense 1"
    slash = "/"
    callCalculate(path)
      

896
smallBursts/Fever Sense 1/Burst003(a).xlsx
124271.42977992853
896
smallBursts/Fever Sense 1/Burst003(b).xlsx
124476.75676062155
896
smallBursts/Fever Sense 1/Burst004(a).xlsx
124174.06064866098
896
smallBursts/Fever Sense 1/Burst004(b).xlsx


KeyboardInterrupt: 